In [1]:
import pandas as pd
import sys
import os
import glob
sys.path.append('../src')
import cargadata as cdat
import datetime
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go

In [2]:
contenido = os.listdir('../data/')
contenido

['1_am_ca',
 '2020_meteorologia_cm.zip',
 '2021_meteorologia_cm.zip',
 '2_cm_ca',
 'calidad_aire_estaciones.csv',
 'descripcion-fichero-open-data-meteorologico.pdf',
 'informacion_estaciones_red_calidad_aire__ayunta.csv',
 'informacion_estaciones_red_calidad_aire__ayunta.geo',
 'zonas_calidad_aire']

In [3]:
con_a2021 = os.listdir('../data/1_am_ca/2020')
con_a2021

['2020_01.csv',
 '2020_02.csv',
 '2020_03.csv',
 '2020_04.csv',
 '2020_05.csv',
 '2020_06.csv',
 '2020_07.csv',
 '2020_08.csv',
 '2020_09.csv',
 '2020_10.csv',
 '2020_11.csv',
 '2020_12.csv']

In [4]:
df = pd.read_csv('../data/1_am_ca/2020/2020_01.csv', sep = ';')

Exploramos el data set suelto

In [73]:
df.head(30)

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,h01,h02,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
0,28,79,4,1,28079004_1_38,2020,1,1,7.0,8.0,...,13.0,12.0,11.0,10.0,10.0,12.0,14.0,12.0,11.0,9.0
1,28,79,4,1,28079004_1_38,2020,1,2,8.0,8.0,...,14.0,12.0,11.0,10.0,11.0,14.0,14.0,15.0,12.0,10.0
2,28,79,4,1,28079004_1_38,2020,1,3,9.0,8.0,...,9.0,10.0,11.0,12.0,12.0,12.0,13.0,13.0,10.0,8.0
3,28,79,4,1,28079004_1_38,2020,1,4,7.0,5.0,...,9.0,9.0,9.0,9.0,10.0,10.0,11.0,10.0,8.0,9.0
4,28,79,4,1,28079004_1_38,2020,1,5,8.0,6.0,...,14.0,11.0,11.0,11.0,11.0,12.0,14.0,13.0,11.0,9.0
5,28,79,4,1,28079004_1_38,2020,1,6,8.0,7.0,...,14.0,12.0,11.0,11.0,12.0,12.0,14.0,15.0,13.0,10.0
6,28,79,4,1,28079004_1_38,2020,1,7,8.0,7.0,...,16.0,14.0,14.0,14.0,17.0,19.0,16.0,18.0,15.0,12.0
7,28,79,4,1,28079004_1_38,2020,1,8,10.0,9.0,...,13.0,11.0,13.0,17.0,18.0,22.0,22.0,20.0,16.0,12.0
8,28,79,4,1,28079004_1_38,2020,1,9,10.0,10.0,...,11.0,11.0,11.0,11.0,12.0,11.0,12.0,12.0,12.0,10.0
9,28,79,4,1,28079004_1_38,2020,1,10,9.0,9.0,...,10.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,9.0,8.0


In [81]:
df[df['estacion']==4].magnitud.unique()

array([ 1,  6,  7,  8, 12])

In [82]:
df[df['estacion']==8].magnitud.unique()

array([ 1,  6,  7,  8,  9, 10, 12, 14, 20, 30, 35, 42, 43, 44])

In [83]:
df.magnitud.unique()

array([ 1,  6,  7,  8, 12,  9, 10, 14, 20, 30, 35, 42, 43, 44])

In [84]:
df.estacion.unique()

array([ 4,  8, 11, 16, 17, 18, 24, 27, 35, 36, 38, 39, 40, 47, 48, 49, 50,
       54, 55, 56, 57, 58, 59, 60])

Vamos a gestionara los datos marcados como no válidos. si son no validos los voy a igualar al anterior si válido

In [6]:
df[f'V01'].value_counts()

V    4678
N      15
Name: V01, dtype: int64

Pacece que cada columna suel tener alrederor de 20 valores así


Primero voy a convertir estos valores en Nan;

In [7]:
horas = ['01', '02','03','04','05', '06', '07', '08', '09', '10', '11', '12',
         '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']

In [8]:
columns_names = ['provincia', 'municipio', 'estacion', 'magnitud', 'punto_muestreo',
       'year', 'month', 'day', 'h01', 'v01', 'h02', 'v02', 'h03', 'v03', 'h04',
       'v04', 'h05', 'v05', 'h06', 'v06', 'h07', 'v07', 'h08', 'v08', 'h09',
       'v09', 'h10', 'v10', 'h11', 'v11', 'h12', 'v12', 'h13', 'v13', 'h14',
       'v14', 'h15', 'v15', 'h16', 'v16', 'h17', 'v17', 'h18', 'v18', 'h19',
       'v19', 'h20', 'v20', 'h21', 'v21', 'h22', 'v22', 'h23', 'v23', 'h24',
       'v24']

paso a minusculas las colunas 

In [9]:
df.set_axis(columns_names, axis='columns', inplace=True)

In [10]:
df.sample()
    

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,h01,v01,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
491,28,79,8,43,28079008_43_2,2020,1,10,1.46,V,...,1.39,V,1.4,V,1.4,V,1.4,V,1.4,V


Aplico la magia de Ester😸🐈🐅🐆🐅 a todas mis columnas

In [11]:
for hora in horas:
    df[f'h{hora}'] = np.where(df[f'v{hora}']=='N', np.nan, df[f'h{hora}'])

Compruebo que efectivamente se ha puesto los nan donde corresponde.

In [12]:
for hora in horas:
    print(f'NaN en columna v{hora} =' ,df[f'v{hora}'].value_counts()[1],f'------->valores nan en la columna h{hora} =',df[f'h{hora}'].isna().sum())
    print()

NaN en columna v01 = 15 ------->valores nan en la columna h01 = 15

NaN en columna v02 = 17 ------->valores nan en la columna h02 = 17

NaN en columna v03 = 22 ------->valores nan en la columna h03 = 22

NaN en columna v04 = 18 ------->valores nan en la columna h04 = 18

NaN en columna v05 = 21 ------->valores nan en la columna h05 = 21

NaN en columna v06 = 30 ------->valores nan en la columna h06 = 30

NaN en columna v07 = 17 ------->valores nan en la columna h07 = 17

NaN en columna v08 = 20 ------->valores nan en la columna h08 = 20

NaN en columna v09 = 47 ------->valores nan en la columna h09 = 47

NaN en columna v10 = 86 ------->valores nan en la columna h10 = 86

NaN en columna v11 = 78 ------->valores nan en la columna h11 = 78

NaN en columna v12 = 91 ------->valores nan en la columna h12 = 91

NaN en columna v13 = 111 ------->valores nan en la columna h13 = 111

NaN en columna v14 = 104 ------->valores nan en la columna h14 = 104

NaN en columna v15 = 69 ------->valores nan 

In [13]:
df.sample()

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,h01,v01,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
834,28,79,16,12,28079016_12_8,2020,1,12,221.0,V,...,206.0,V,202.0,V,237.0,V,405.0,V,292.0,V


Dropeo las columnas de verificacion

In [14]:
li_v = []
for hora in horas:
    li_v.append(f'v{hora}')    
li_v

['v01',
 'v02',
 'v03',
 'v04',
 'v05',
 'v06',
 'v07',
 'v08',
 'v09',
 'v10',
 'v11',
 'v12',
 'v13',
 'v14',
 'v15',
 'v16',
 'v17',
 'v18',
 'v19',
 'v20',
 'v21',
 'v22',
 'v23',
 'v24']

In [15]:
df.drop(li_v, axis = 1, inplace=True)

In [16]:
df.sample()

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,h01,h02,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
2402,28,79,38,9,28079038_9_47,2020,1,30,3.0,4.0,...,11.0,10.0,7.0,6.0,7.0,8.0,10.0,9.0,6.0,6.0


Aplico el valor anterior inmediatamente válido con un método de propagacion de ffill  

In [17]:
for hora in horas:
    print(df[f'h{hora}'].isna().sum())

15
17
22
18
21
30
17
20
47
86
78
91
111
104
69
44
71
66
38
39
28
18
20
20


In [18]:
df.fillna(method='ffill', inplace=True)

In [19]:
for hora in horas:
    print(df[f'h{hora}'].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [20]:
4693 * 24

112632

In [21]:
li_h = []
for hora in horas:
    li_h.append(f'h{hora}')  
li_h


['h01',
 'h02',
 'h03',
 'h04',
 'h05',
 'h06',
 'h07',
 'h08',
 'h09',
 'h10',
 'h11',
 'h12',
 'h13',
 'h14',
 'h15',
 'h16',
 'h17',
 'h18',
 'h19',
 'h20',
 'h21',
 'h22',
 'h23',
 'h24']

In [22]:
df.columns

Index(['provincia', 'municipio', 'estacion', 'magnitud', 'punto_muestreo',
       'year', 'month', 'day', 'h01', 'h02', 'h03', 'h04', 'h05', 'h06', 'h07',
       'h08', 'h09', 'h10', 'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17',
       'h18', 'h19', 'h20', 'h21', 'h22', 'h23', 'h24'],
      dtype='object')

In [23]:
df_melt = df.melt(['provincia', 'municipio', 'estacion', 'magnitud', 'punto_muestreo',
       'year', 'month', 'day']) 
df_melt.sample()

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,variable,value
62734,28,79,24,43,28079024_43_2,2020,1,4,h14,1.06


In [24]:
df_melt.rename(columns = {'variable':'hour'}, inplace = True)

In [25]:
df_melt.head(49)

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,hour,value
0,28,79,4,1,28079004_1_38,2020,1,1,h01,7.0
1,28,79,4,1,28079004_1_38,2020,1,2,h01,8.0
2,28,79,4,1,28079004_1_38,2020,1,3,h01,9.0
3,28,79,4,1,28079004_1_38,2020,1,4,h01,7.0
4,28,79,4,1,28079004_1_38,2020,1,5,h01,8.0
5,28,79,4,1,28079004_1_38,2020,1,6,h01,8.0
6,28,79,4,1,28079004_1_38,2020,1,7,h01,8.0
7,28,79,4,1,28079004_1_38,2020,1,8,h01,10.0
8,28,79,4,1,28079004_1_38,2020,1,9,h01,10.0
9,28,79,4,1,28079004_1_38,2020,1,10,h01,9.0


In [26]:
li_h; 

In [27]:
horas = [int(x[1:]) for x in li_h]
horas;

In [28]:
dic_horas = dict(zip(li_h, horas))

In [29]:
dic_horas

{'h01': 1,
 'h02': 2,
 'h03': 3,
 'h04': 4,
 'h05': 5,
 'h06': 6,
 'h07': 7,
 'h08': 8,
 'h09': 9,
 'h10': 10,
 'h11': 11,
 'h12': 12,
 'h13': 13,
 'h14': 14,
 'h15': 15,
 'h16': 16,
 'h17': 17,
 'h18': 18,
 'h19': 19,
 'h20': 20,
 'h21': 21,
 'h22': 22,
 'h23': 23,
 'h24': 24}

In [30]:
df_melt['hour']= df_melt['hour'].map(dic_horas)

In [31]:
df_melt.tail(40)

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,hour,value
112592,28,79,60,12,28079060_12_8,2020,1,23,24,26.00
112593,28,79,60,12,28079060_12_8,2020,1,24,24,33.00
112594,28,79,60,12,28079060_12_8,2020,1,25,24,24.00
112595,28,79,60,12,28079060_12_8,2020,1,26,24,27.00
112596,28,79,60,12,28079060_12_8,2020,1,27,24,4.00
112597,28,79,60,12,28079060_12_8,2020,1,28,24,3.00
112598,28,79,60,12,28079060_12_8,2020,1,29,24,3.00
112599,28,79,60,12,28079060_12_8,2020,1,30,24,16.00
112600,28,79,60,12,28079060_12_8,2020,1,31,24,27.00
112601,28,79,60,14,28079060_14_6,2020,1,1,24,1.00


In [32]:
df_melt['miDt']=pd.to_datetime(df_melt[['year','month','day','hour']])

In [33]:
df_melt

,provincia,municipio,estacion,magnitud,punto_muestreo,year,month,day,hour,value,miDt
0,28,79,4,1,28079004_1_38,2020,1,1,1,7.00,2020-01-01 01:00:00
1,28,79,4,1,28079004_1_38,2020,1,2,1,8.00,2020-01-02 01:00:00
2,28,79,4,1,28079004_1_38,2020,1,3,1,9.00,2020-01-03 01:00:00
3,28,79,4,1,28079004_1_38,2020,1,4,1,7.00,2020-01-04 01:00:00
4,28,79,4,1,28079004_1_38,2020,1,5,1,8.00,2020-01-05 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...
112627,28,79,60,14,28079060_14_6,2020,1,27,24,65.94,2020-01-28 00:00:00
112628,28,79,60,14,28079060_14_6,2020,1,28,24,58.76,2020-01-29 00:00:00
112629,28,79,60,14,28079060_14_6,2020,1,29,24,57.38,2020-01-30 00:00:00
112630,28,79,60,14,28079060_14_6,2020,1,30,24,66.70,2020-01-31 00:00:00


In [34]:
df_melt.drop(['provincia','year','month','day','hour'], axis = 1, inplace=True)

In [35]:
df_melt.sample()

,municipio,estacion,magnitud,punto_muestreo,value,miDt
8881,79,57,7,28079057_7_8,1.0,2020-01-23 02:00:00


In [36]:
df_melt = df_melt.iloc[:,[0,1,2,3,5,4]]

In [56]:
df_melt.head(50)

,municipio,estacion,magnitud,punto_muestreo,miDt,value
0,79,4,1,28079004_1_38,2020-01-01 01:00:00,7.0
1,79,4,1,28079004_1_38,2020-01-02 01:00:00,8.0
2,79,4,1,28079004_1_38,2020-01-03 01:00:00,9.0
3,79,4,1,28079004_1_38,2020-01-04 01:00:00,7.0
4,79,4,1,28079004_1_38,2020-01-05 01:00:00,8.0
5,79,4,1,28079004_1_38,2020-01-06 01:00:00,8.0
6,79,4,1,28079004_1_38,2020-01-07 01:00:00,8.0
7,79,4,1,28079004_1_38,2020-01-08 01:00:00,10.0
8,79,4,1,28079004_1_38,2020-01-09 01:00:00,10.0
9,79,4,1,28079004_1_38,2020-01-10 01:00:00,9.0


In [58]:
df_melt['magnitud'].unique()

array([ 1,  6,  7,  8, 12,  9, 10, 14, 20, 30, 35, 42, 43, 44])

In [68]:
df_total = pd.pivot_table(data=df_melt,
               index=['miDt', 'estacion', 'municipio'],
               columns=['magnitud']
              )

In [71]:
df_total.tail(30)

value                               \
magnitud                                  1    6     7     8     9     10   
miDt                estacion municipio                                      
2020-01-31 23:00:00 55       79          NaN  NaN  10.0  46.0   NaN  63.0   
                    56       79          NaN  0.3  18.0  44.0   9.0  11.0   
                    57       79          5.0  0.2   3.0  22.0   NaN   8.0   
                    58       79          NaN  NaN   2.0  23.0   NaN   NaN   
                    59       79          NaN  NaN  10.0  52.0   NaN   NaN   
                    60       79          NaN  NaN   3.0  22.0   NaN   6.0   
2020-02-01 00:00:00 4        79         10.0  0.2   3.0  33.0   NaN   NaN   
                    8        79          6.0  0.4  12.0  46.0   8.0  12.0   
                    11       79          NaN  NaN   2.0  25.0   NaN   NaN   
                    16       79          NaN  0.4   8.0  46.0   NaN   NaN   
                    17       79         10.0  NaN  22.0  28.0   NaN   NaN   
                    18       79          4.0  0.4   4.0  35.0   NaN   6.0   
                    24       79          5.0  0.2   1.0  16.0   4.0   7.0   
                    27       79          NaN  NaN   8.0  61.0   NaN   NaN   
                    35       79          4.0  0.8   4.0  38.0   NaN   NaN   
                    36       79          8.0  0.3   5.0  39.0   NaN  10.0   
                    38       79          2.0  NaN   3.0  21.0   7.0  10.0   
                    39       79          NaN  0.3   2.0  26.0   NaN   NaN   
                    40       79          7.0  NaN   9.0  45.0   NaN  23.0   
                    47       79          NaN  NaN   1.0  31.0   6.0   7.0   
                    48       79          NaN  NaN   5.0  36.0  11.0  14.0   
                    49       79          NaN  NaN   1.0  42.0   NaN   NaN   
                    50       79          NaN  NaN  14.0  25.0   7.0  10.0   
                    54       79          NaN  NaN   2.0  33.0   NaN   NaN   
                    55       79          NaN  NaN   6.0  42.0   NaN  49.0   
                    56       79          NaN  0.3  27.0  47.0   9.0  11.0   
                    57       79          5.0  0.2   2.0  22.0   NaN   5.0   
                    58       79          NaN  NaN   2.0  20.0   NaN   NaN   
                    59       79          NaN  NaN   2.0  45.0   NaN   NaN   
                    60       79          NaN  NaN   3.0  23.0   NaN   9.0   

                                                                          \
magnitud                                  12     14   20   30   35    42   
miDt                estacion municipio                                     
2020-01-31 23:00:00 55       79         61.0    NaN  3.4  1.0  0.5  1.23   
                    56       79         72.0  22.87  NaN  NaN  NaN   NaN   
                    57       79         27.0    NaN  NaN  NaN  NaN   NaN   
                    58       79         26.0  24.80  NaN  NaN  NaN   NaN   
                    59       79         67.0   8.98  NaN  NaN  NaN   NaN   
                    60       79         27.0  43.53  NaN  NaN  NaN   NaN   
2020-02-01 00:00:00 4        79         38.0    NaN  NaN  NaN  NaN   NaN   
                    8        79         65.0   9.43  2.3  0.8  1.3  1.47   
                    11       79         28.0    NaN  0.1  0.2  0.1   NaN   
                    16       79         58.0  10.53  NaN  NaN  NaN   NaN   
                    17       79         62.0  27.56  NaN  NaN  NaN   NaN   
                    18       79         42.0  22.54  1.6  0.4  0.3   NaN   
                    24       79         17.0  29.37  0.6  0.4  0.1  1.15   
                    27       79         73.0   2.71  NaN  NaN  NaN   NaN   
                    35       79         45.0  16.41  NaN  NaN  NaN   NaN   
                    36       79         46.0    NaN  NaN  NaN  NaN   NaN   
                    38       79         26.0    NaN  